In [5]:
import torch
import numpy as np

import sys
from pathlib import Path

sys.path.append('/home/nikita/e2e-driving')

import time
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torch.distributions import Categorical
import math

from ibc import optimizers
from dataloading.nvidia import NvidiaDataset
from metrics.metrics import calculate_open_loop_metrics
from pilotnet import PilotnetEBM
import pandas as pd


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    all_predictions = []
    inference_times = []
    progress_bar = tqdm(total=len(dataloader), smoothing=0)

    epoch_mae = 0.0
    epoch_entropy = 0.0
    ask_batch_timestamp = time.time()
    for i, (input, target, _) in enumerate(dataloader):
        recv_batch_timestap = time.time()

        inputs = input['image'].to(device)
        target = target.to(device, torch.float32)

        inference_start = time.perf_counter()
        preds, energy = model(inputs)
        inference_end = time.perf_counter()

        inference_time = inference_end - inference_start
        inference_times.append(inference_time)

        mae = F.l1_loss(preds, target.view(-1, 1))
        mae_degrees = math.degrees(mae.item())
        epoch_mae += mae_degrees

        entropy = Categorical(F.softmax(-energy, dim=-1)).entropy().mean() / math.log(energy.shape[-1])
        epoch_entropy += entropy.item()

        all_predictions.extend(preds.cpu().squeeze().numpy())

        progress_bar.update(1)
        progress_bar.set_description(f'MAE: {(epoch_mae / (i + 1)):.4f} | Entropy: {(epoch_entropy / (i + 1)):.4f}')

        ask_batch_timestamp = time.time()

    avg_mae = epoch_mae / len(dataloader)
    avg_entropy = epoch_entropy / len(dataloader)
    result = np.array(all_predictions)
    return avg_mae, avg_entropy, result


def calculate_metrics(fps, predictions, valid_loader):
    '''For steering angle only.'''

    frames_df = valid_loader.dataset.frames
    true_steering_angles = frames_df.steering_angle.to_numpy()
    metrics = calculate_open_loop_metrics(predictions, true_steering_angles, fps=fps)
    return metrics

/home/nikita/miniconda3/envs/e2e/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
from sklearn.neighbors import BallTree

class VelocityModel:
    def __init__(self, positions_parquet='positions.parquet', vector_velocity=30):
        self.vector_velocity = vector_velocity
        self.positions_df = pd.read_parquet(positions_parquet)
        self.tree = BallTree(self.positions_df[["position_x", "position_y", "position_x2", "position_y2"]])

    def find_closest_position(self, x, y, yaw):
        x2 = x + (self.vector_velocity * np.cos(yaw))
        y2 = y + (self.vector_velocity * np.sin(yaw))

        closest = self.tree.query([[x, y, x2, y2]])
        distance = closest[0][0][0]
        index = closest[1][0][0]
        return self.positions_df.iloc[index], distance

def trim_human_datasets_to_auto_datasets(human_drive_ds, forward_auto_frames, back_auto_frames):
    velmodel = VelocityModel('~/ros-e2e-workspace/src/e2e_platform/config/speed_models/summer2021-positions.parquet')

    forward_start = forward_auto_frames.iloc[0]
    forward_end = forward_auto_frames.iloc[-1]
    back_start = back_auto_frames.iloc[0]
    back_end = back_auto_frames.iloc[-1]

    forward_start_frame, forward_start_dist = velmodel.find_closest_position(forward_start['position_x'], forward_start['position_y'], forward_start['yaw'])
    forward_end_frame, forward_end_dist = velmodel.find_closest_position(forward_end['position_x'], forward_end['position_y'], forward_end['yaw'])
    back_start_frame, back_start_dist = velmodel.find_closest_position(back_start['position_x'], back_start['position_y'], back_start['yaw'])
    back_end_frame, back_end_dist = velmodel.find_closest_position(back_end['position_x'], back_end['position_y'], back_end['yaw'])

    # print('forward start:', forward_start_dist, forward_start_frame.name)
    # print('forward end:', forward_end_dist, forward_end_frame.name)
    # print('back start:', back_start_dist, back_start_frame.name)
    # print('back end:', back_end_dist, back_end_frame.name)

    human_indices = human_drive_ds.frames.index.to_numpy()
    human_indices_current = human_indices[:-1]
    human_indices_next = human_indices[1:]
    change_of_direction_idx = (human_indices_next < human_indices_current).nonzero()[0].item()

    forward_filtered = human_drive_ds.frames.iloc[forward_start_frame.name:forward_end_frame.name]
    back_filtered = human_drive_ds.frames.iloc[change_of_direction_idx+back_start_frame.name:change_of_direction_idx+back_end_frame.name]

    merged = pd.concat([forward_filtered, back_filtered], ignore_index=True).reset_index(drop=True)
    human_drive_ds.frames = merged

In [6]:
human_drive = Path('/data/Bolt/dataset-new-big/summer2021/2021-10-26-10-49-06_e2e_rec_ss20_elva')
human_drive_back = Path('/data/Bolt/dataset-new-big/summer2021/2021-10-26-11-08-59_e2e_rec_ss20_elva_back')
human_drive_ds = NvidiaDataset([human_drive, human_drive_back])

# trim human dataset to auto dataset
forward_frames = pd.read_csv('/data/Bolt/drives-nikita-thesis/2022-08-04-17-33-33_elva_forward_ebm_regularized_v3/nvidia_frames.csv')
back_frames = pd.read_csv('/data/Bolt/drives-nikita-thesis/2022-08-04-17-21-58_elva_back_ebm_regularized_v3/nvidia_frames.csv')

trim_human_datasets_to_auto_datasets(human_drive_ds, forward_frames, back_frames)

dataloader = torch.utils.data.DataLoader(human_drive_ds.frames, batch_size=256, shuffle=False, num_workers=8)

/data/Bolt/dataset-new-big/summer2021/2021-10-26-10-49-06_e2e_rec_ss20_elva: length=33045, filtered=0
/data/Bolt/dataset-new-big/summer2021/2021-10-26-11-08-59_e2e_rec_ss20_elva_back: length=33281, filtered=0


In [38]:
pt_model_path = '/home/nikita/e2e-driving/models/20220618185552_steering-angle/last.pt'

def evaluate_model(constant_actions=False, **kwargs):
  model = PilotnetEBM()
  inference_model = optimizers.DFOptimizerConst if constant_actions else optimizers.DFOptimizer
  model = inference_model(model, optimizers.DerivativeFreeConfig(**kwargs))
  model.load_state_dict(torch.load(pt_model_path))
  model.to(device)
  mae, entropy, preds = evaluate(model, valid_loader)

  return mae, entropy, preds

In [19]:
torch.log(torch.tensor(1024))

tensor(6.9315)

In [7]:
inputs = iter(valid_loader).next()[0]['image'][0].unsqueeze(0).to(device)
print(inputs.shape)

torch.Size([1, 3, 68, 264])


In [39]:
import itertools
import json
import wandb

fps = 30

samples = [1024]
iters = [0]
constant_actions = [True]

run_hparams = list(itertools.product(samples, iters, constant_actions))

for idx, (samples, iters, constant_actions) in enumerate(run_hparams):
    # wandb.init(project="ibc-tuning", name=f'{samples}s{iters}it', config={"samples": samples, "iters": iters, "model_path": pt_model_path, "constant_actions": constant_actions})

    print(f'running experiment {idx+1}/{len(run_hparams)}. samples: {samples}, iters: {iters}')
    mae, entropy, preds = evaluate_model(constant_actions=constant_actions, inference_samples=samples, iters=iters)
    metrics = calculate_metrics(fps, preds, valid_loader)
    metrics["entropy"] = entropy
    # wandb.log(metrics)
    print('experiment metrics:')
    print(json.dumps(str(metrics), indent=2))
    print()
    # wandb.finish()

running experiment 1/1. samples: 1024, iters: 0


  0%|          | 0/869 [00:00<?, ?it/s]

experiment metrics:
"{'mae': 8.361593943192089, 'rmse': 33.65489972678105, 'max': 921.8890671983361, 'whiteness': 634.46564, 'expert_whiteness': 25.61257234059799, 'left_mae': 42.129001539884996, 'straight_mae': 5.378298346175109, 'right_mae': 78.43601723044296, 'entropy': 0.4784948651960722}"



: 